In [ ]:
from openai import OpenAI
import concurrent.futures
import pandas as pd
import tiktoken
import random
import textwrap
import re
import json
import time
from PIL import Image, ImageEnhance, ImageOps
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

In [ ]:
class ImageToAscii:
    def __init__(self, char_set='simple', width=64, height=64, density=1.0):
        self.char_set = char_set
        self.width = width
        self.height = height
        self.density = density
        self.ascii_char_sets = {
            'simple': '@%#*+=-:. ',
            'complex': '$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,"^`\'. ',
            'blocks': '█▓▒░ ',
        }

    def preprocess(self, image, brightness=1.0, saturation=1.0, sharpness=1.0, contrast=1.0, hue=0.0):
        """
        Apply preprocessing to the image such as adjusting brightness, saturation, sharpness, contrast, and hue.
        """
        if brightness != 1.0:
            enhancer = ImageEnhance.Brightness(image)
            image = enhancer.enhance(brightness)
        
        if saturation != 1.0:
            enhancer = ImageEnhance.Color(image)
            image = enhancer.enhance(saturation)
        
        if sharpness != 1.0:
            enhancer = ImageEnhance.Sharpness(image)
            image = enhancer.enhance(sharpness)
        
        if contrast != 1.0:
            enhancer = ImageEnhance.Contrast(image)
            image = enhancer.enhance(contrast)
        
        if hue != 0.0:
            pass

        return image

    def resize_image(self, image):
        return image.resize((self.width, self.height))

    def adjust_space_density(self, line):
        white_space = ' ' * int(1 / self.density)
        return white_space.join(line)

    def pixels_to_chars(self, image):
        pixels = np.array(image)
        chars = "".join([self.ascii_char_sets[self.char_set][pixel * len(self.ascii_char_sets[self.char_set]) // 256] 
                         for pixel in pixels.flatten()])
        return chars

    def generate(self, image_path, brightness=1.0, saturation=1.0, sharpness=1.0, contrast=1.0, hue=0.0):
        image = Image.open(image_path)
        image = self.preprocess(image, brightness, saturation, sharpness, contrast, hue)
        image = self.resize_image(image)
        image = image.convert("L")  # Convert to grayscale
        chars = self.pixels_to_chars(image)
        ascii_art = "\n".join([self.adjust_space_density(chars[index:index + self.width]) 
                               for index in range(0, len(chars), self.width)])
        return ascii_art

In [ ]:
class GPT:
    def __init__(self, openai_client):
        self.openai_client = openai_client
        self.total_tokens = 0
        self.cost = 0.0
    
    def num_tokens(self, string: str, encoding_name: str) -> int:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def gpt(self, prompt, max_tokens, temp, model):
        response = self.openai_client.chat.completions.create(
            model=model,
            messages=prompt,
            temperature=temp,
            max_tokens=max_tokens,
        )
        response = response.choices[0].message.content
        prompt_tokens = self.num_tokens(str(prompt), "cl100k_base")
        response_tokens = self.num_tokens(str(response), "cl100k_base")

        pricing = {
            'gpt-4-0125-preview': {'input': 0.01, 'output': 0.03},
            'gpt-4-1106-preview': {'input': 0.01, 'output': 0.03},
            'gpt-4': {'input': 0.03, 'output': 0.06},
            'gpt-4-32k': {'input': 0.06, 'output': 0.12},
            'gpt-3.5-turbo-0125': {'input': 0.0005, 'output': 0.0015},
            'gpt-3.5-turbo': {'input': 0.0030, 'output': 0.0060},
        }

        rates = pricing.get(model, {'input': 0.0, 'output': 0.0})

        self.cost += (prompt_tokens * rates['input'] + response_tokens * rates['output']) / 1000
        self.total_tokens += prompt_tokens + response_tokens

        return response

    def dalle(self, prompt, img_size, model):
        pricing = {
            ('dall-e-3', '1024x1024'): 0.040,
            ('dall-e-3', '1024x1792'): 0.080,
            ('dall-e-3', '1792x1024'): 0.080,
            ('dall-e-2', '1024x1024'): 0.020,
            ('dall-e-2', '512x512'): 0.018,
            ('dall-e-2', '256x256'): 0.016,
        }
        cost_per_image = pricing.get((model, img_size), 0.0)

        

        response = self.openai_client.images.generate(
            model=model,
            prompt=prompt,
            size=img_size,
            n=1,
        )
        print(prompt)

        self.cost += cost_per_image
        return response.data[0].url
    
    def gen_img_sequence(self, random_sequence, img_size, model):
        prompts = []
        for item in random_sequence:
            prompt = [
                {"role": "user", "content": textwrap.dedent(f"""\
                Instruction: Your task is optimize user's input for image generation.
                Example User Input: Create an image of a single cat on a blank background. The background is completely white, emphasizing the cat's features.
                Example Output: A single cat sitting on a completely white background, emphasizing its distinct features. The cat is fluffy with a rich, thick coat, displaying a variety of colors such as ginger, black, and white. Its eyes are large and expressive, colored a deep emerald green. The cat's pose is relaxed, with its tail curled around its paws.
                User Input: Create an image of a single {item} on a blank background. The background is completely white, emphasizing the {item}'s features.
                Output: 
                """)
                }
            ]
            prompts.append(prompt)

        # optimize the prompt for better generation
        optimized_prompts = []
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(self.gpt, prompt, 128, 0.7, 'gpt-4') for prompt in prompts]
            for future in as_completed(futures):
                optimized_prompts.append(future.result())

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(self.dalle, prompt, img_size, model) for prompt in optimized_prompts]
            results = []
            for future in as_completed(futures):
                results.append(future.result())
            return results

    def extract_after_keyword(self, text, keyword="keyword"):
        cleaned_text = re.sub(r'\s+', ' ', text)

        # Pattern to match the keyword case-insensitively and capture the text after it
        pattern = re.compile(r'\b' + re.escape(keyword) + r':\s*(.*)', re.IGNORECASE)

        # Search for the pattern in the cleaned text
        match = pattern.search(cleaned_text)

        # If a match is found, return the captured group, otherwise return None
        return match.group(1) if match else None

    def gen_random_pair(self, examples):
        prompt = [
            {"role": "user", "content": textwrap.dedent(f"""\
            Instruction: Generate a Python list containing two visually simple and unrelated items. Each entity in the list should be distinct from each other and easily recognizable.
            Examples: 
            ["apple", "egg"]
            {examples}
            You Output (one python list that contains two random entities): 
            """)
            }
        ]
        answer = self.gpt(prompt, 128, 0.7, 'gpt-4')
        return answer

    def gen_random_sequence(self, random_pair):
        while True:
            sequence = []
            for _ in range(4):
                sequence.append(random.choice(random_pair))
            if len(set(sequence)) > 1:
                return sequence

    def download_and_save_images(self, urls, folder_name):
        os.makedirs(folder_name, exist_ok=True)
        filenames = ['A.png', 'B.png', 'C.png', 'D.png']

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = []
            for url, filename in zip(urls, filenames):
                path = os.path.join(folder_name, filename)
                futures.append(executor.submit(self.download_image, url, path))

            for future in as_completed(futures):
                future.result()

    def download_image(self, url, path):
        response = requests.get(url)
        response.raise_for_status()  
        with open(path, 'wb') as f:
            f.write(response.content)



openai_client = OpenAI(api_key='none')
model = GPT(openai_client)

In [49]:
# generate random image captcha
n_samples = 100

previous_samples = []
for i in range(n_samples):
    previous_10_samples = previous_samples[-10:]
    prompt_examples = ""
    for sample in previous_10_samples:
        prompt_examples += sample+"\n"

    random_pair = model.gen_random_pair(prompt_examples)
    previous_samples.append(random_pair)

    random_pair = json.loads(random_pair)
    random_sequence = model.gen_random_sequence(random_pair)
    img_urls = model.gen_img_sequence(random_sequence, '1024x1024', 'dall-e-3')

    folder = f"samples/{'-'.join(random_pair)}"
    model.download_and_save_images(img_urls, folder)
    with open(folder+'/obj.json', 'w') as file:
        json.dump(random_sequence, file, indent=4)

A single car showcased on a completely white background, accentuating its distinct features. The car is sleek, with a glossy, polished finish that reflects light brilliantly. It sports a vibrant, red color that contrasts starkly against the white backdrop. The car's design is aerodynamic, with curved lines and an overall streamlined shape. Its headlights are bright and clear, while the wheels are large and sturdy. The car appears luxurious and high-end, highlighting the elegance of automotive design.
A single car positioned on a completely white background, emphasizing its striking features. The car is sleek and polished, with a glossy finish that reflects light beautifully. It displays a vibrant, red hue, which contrasts dramatically against the white backdrop. The car's design is aerodynamic, with smooth, curvaceous lines and polished chrome detailing. Its wheels are large and robust, giving it a sturdy and powerful stance. The headlights are bright and clear, adding to the overall a

In [ ]:
root_dir = "./samples"
converter = ImageToAscii(char_set='simple', width=64, height=64, density=2)
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for image_file in os.listdir(subdir_path):
            if image_file.endswith('.png'):
                file_path = os.path.join(subdir_path, image_file)
                ascii_art = converter.generate(file_path)
                text_file = os.path.splitext(file_path)[0] + '.txt'
                with open(text_file, 'w') as f:
                    f.write(ascii_art)
                print(f"Generated ASCII art for {image_file}")

In [ ]:
# ascii select
